In [2]:
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate
from matplotlib.ticker import PercentFormatter

In [3]:
plt.rcParams["font.sans-serif"] = "Source Han Sans SC"
plt.rcParams["figure.dpi"] = 300
plt.rcParams["savefig.bbox"] = "tight"

In [4]:
# 描边预设
stroke_white = [pe.withStroke(linewidth=2.5, foreground="white")]
stroke_black = [pe.withStroke(linewidth=2.5, foreground="black")]

In [5]:
df = pd.read_csv("后勤分队源石锭电表倒转成功概率.csv", index_col=["厉钱数量", "初始源石锭数量"])

In [ ]:
厉钱数量范围 = range(5, 9+1)
x_max = 1024

fig, ax = plt.subplots(figsize=(10, 6))
colors = [f"C{i}" for i in range(len(厉钱数量范围))]

# 绘制参考线
ref_lines = [0.25, 0.5, 0.75, 0.9]
for y_ref in ref_lines:
    ax.axhline(y_ref, color="gray", linestyle="--", path_effects=stroke_white, zorder=2)
    ax.annotate(f"{y_ref:.0%}", (x_max, y_ref),
                ha="right", va="center", xytext=(-5, 0), textcoords="offset points",
                color="gray", fontweight="bold", path_effects=stroke_white, zorder=40)

for i, 厉钱数量 in enumerate(厉钱数量范围):
    data = df.loc[厉钱数量]
    x = data.index.values
    y = data["成功概率"].values

    spline = scipy.interpolate.make_interp_spline(x, y)
    x_new = np.arange(0, 1024+1, 1)
    y_smooth = spline(x_new)
    ax.plot(x_new, y_smooth,
            label=f"厉钱数量＝{厉钱数量}",
            color=colors[i], path_effects=stroke_white, zorder=20)

    # 计算并标注所有交点
    for y_ref in ref_lines:
        idx = np.searchsorted(y_smooth, y_ref)
        if idx == len(y_smooth):
            continue
        x_cross = x_new[idx]
        ax.scatter(x_cross, y_ref, marker=".",
                   s=10, color=colors[i], path_effects=stroke_white, zorder=30)
        # 奇数在上，偶数在下
        if 厉钱数量 % 2 == 1:
            offset = (0, 2)
            va = "baseline"
        else:
            offset = (0, -2)
            va = "top"
        ax.annotate(f"{x_cross}", (x_cross, y_ref),
                    ha="right", va=va, xytext=offset, textcoords="offset points",
                    color=colors[i], fontweight="bold", path_effects=stroke_white, zorder=40)

ax.set_title("后勤分队电表倒转成功概率", fontweight="bold")
ax.set_xlabel("初始源石锭数量", fontweight="bold")
ax.set_ylabel("成功概率", fontweight="bold")

ax.minorticks_on()
ax.grid(True, which="major", linewidth=1.2)
ax.grid(True, which="minor", linewidth=0.6)
ax.yaxis.set_major_formatter(PercentFormatter(1.0))

ax.set_xlim(0, x_max)
ax.set_ylim(bottom=0)

ax.legend(loc="upper left")

fig.savefig("后勤分队源石锭电表倒转成功概率.png")